**Run Flask on Colab**

In [ ]:
!pip install flask-ngrok
!pip install gevent
!pip install pandasql

     |████████████████████████████████| 5.6MB 8.0MB/s 
     |████████████████████████████████| 256kB 40.8MB/s 
  Created wheel for pandasql: filename=pandasql-0.7.3-cp37-none-any.whl size=26784 sha256=8401f3e5b25bc02c56e2c8d81988c01429879ce758805e8f4521fc8c5cafa602
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [ ]:
'''
from flask_ngrok import run_with_ngrok
from flask import Flask

# # Running the flask app
app = Flask(__name__)

# # start ngrok when app is run
run_with_ngrok(app)

@app.route("/")
def index():
  return "<h1>Home Page!! </h1>"

@app.route("/about")
def about():
  return "<h1>About Page!! </h1>"

@app.route("/us")
def us():
  return "<h1>Us Page!! </h1>"

app.run()

'''

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://368516ac4bee.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
# To unzip main files
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/Colab Notebooks/Project_Files.zip"
# file_name = "/content/drive/MyDrive/Colab Notebooks/Copy of final_1_basicfiles.zip"
print(file_name)
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

/content/drive/MyDrive/Colab Notebooks/Project_Files.zip
done


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# # To unzip model files

# from zipfile import ZipFile
# #file_name = "/content/drive/MyDrive/Colab Notebooks/movie_model_cnn.zip"
# file_name = "/content/drive/MyDrive/Colab Notebooks/model_CNN-01-0.393-0.324_(0-98).zip"
# with ZipFile(file_name, 'r') as zip:
#   zip.extractall()
#   print('done')


In [ ]:
from __future__ import division, print_function
# coding=utf-8
import sys, os, glob, re, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# !pip install pandasql
import pandasql as ps

# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
# from tensorflow.keras.preprocessing import image
from keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer
from flask_ngrok import run_with_ngrok

import warnings
warnings.filterwarnings('ignore')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')
nltk.download('stopwords')

time.sleep(5)

# Define a flask app
app = Flask(__name__)

# start ngrok when app is run
run_with_ngrok(app)

# Model saved with Keras model.save()
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/newCnn.h5'

# Load your trained model
model = load_model(MODEL_PATH)
# model._make_predict_function()          # Necessary
print('\nModel loaded. Start serving...')
print('\nModel loaded. Check http://127.0.0.1:5000/')


# to find similarity
def simi(x,y):
    X =x
    Y =y
    # tokenization
    # associate a number to word
    X_list = word_tokenize(X.lower())
    Y_list = word_tokenize(Y.lower())

    # sw contains the list of stopwords
    # stopwords - is , a , the etc.
    sw = stopwords.words('english')
    l1 =[];l2 =[]

    # remove stop words from the string
    X_set = {w for w in X_list if not w in sw}
    Y_set = {w for w in Y_list if not w in sw}

    # form a set containing keywords of both strings
    rvector = X_set.union(Y_set)
    for w in rvector:
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0)
        if w in Y_set: l2.append(1)
        else: l2.append(0)
    c = 0

    # cosine formula
    for i in range(len(rvector)):
        c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)

    return round(cosine,6)

def model_predict(img_path, model, df):
    #allName = ['Drama', 'Comedy', 'Romance', 'Action', 'Adventure', 'Thriller', 'Western', 'Science Fiction', 'Fantasy', 'Horror', 'Family', 'Crime', 'War', 'Mystery', 'Animation', 'Music', 'Documentary', 'History']
    allName =['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime','Documentary', 'Drama', 'Family', 'Fantasy', 'History',
              'Horror','Music', 'Musical', 'Mystery', 'N/A', 'News', 'Reality-TV','Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War',
              'Western']
    # SIZE = (150, 101, 3)
    # SIZE = (224, 224, 3)
    SIZE = (300, 300, 3)

    img = image.load_img(img_path, target_size=(SIZE[0], SIZE[1], 3))
    plt.imshow(img)
    img = image.img_to_array(img)
    img = img/255.0

    img = img.reshape(1, SIZE[0], SIZE[1], 3)

    y_prob = model.predict(img)
    print("y_prob : " , y_prob)
    top3 = np.argsort(y_prob[0])[-3:]
    print('top3 : ',top3)
    li=[]
    for i in range(3):
        li.append(allName[top3[i]])
    li.reverse()

    print("Li " , li)

    val = []
    for i in range(len(df)):
        val.append(simi(str(li[0]+' '+li[1]+' '+li[2]),re.sub('[^a-zA-Z0-9 \n\.]', ' ', str(df.genres[i]))))

    print("Val ", val)
    return str(li), val

@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')

@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        file_path = os.path.join('static', secure_filename(f.filename))
        # Save the file to ./uploads
        f.save(file_path)

        df = pd.read_csv(r'allmoviefrom134.csv',encoding="utf8")

        # Make prediction
        preds, val = model_predict(file_path, model, df)
        result = str(preds)

        df['similarity'] = val

        test = df.sort_values('similarity', ascending=False)
        test.reset_index(inplace = True)

        test2 = ps.sqldf("select * from test limit 10")
        lila = []
        for row in test2.itertuples():
            lila.append(row)

        print("Lila " ,lila)
        return render_template('index.html', result=result, file_path=file_path[7:],test2=lila)
    return None

if __name__ == '__main__':
    # app.run(debug=True)
    app.run()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.

Model loaded. Start serving...

Model loaded. Check http://127.0.0.1:5000/
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://71bebf1201ec.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [17/Jun/2021 05:02:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2021 05:02:42] "GET /favicon.ico HTTP/1.1" 404 -


y_prob :  [[2.2192477e-01 1.7216811e-01 1.5613932e-02 4.8583858e-02 1.4902984e-01
  1.4038226e-01 6.9312878e-02 4.1750082e-01 5.3673729e-02 1.5344699e-01
  3.1473082e-02 2.4161127e-01 2.6922433e-02 1.4481691e-02 1.5292522e-01
  1.3449956e-03 1.3945637e-03 3.9629423e-04 6.0476933e-02 1.4501894e-01
  1.7132998e-02 1.0984170e-02 1.8275681e-01 1.5757937e-02 3.6170357e-03]]
top3 :  [ 0 11  7]
Li  ['Drama', 'Horror', 'Action']
Val  [0.0, 0.0, 0.333333, 0.0, 0.57735, 0.0, 0.57735, 0.333333, 0.333333, 0.333333, 0.0, 0.408248, 0.408248, 0.408248, 0.408248, 0.0, 0.0, 0.333333, 0.0, 0.408248, 0.288675, 0.258199, 0.408248, 0.57735, 0.333333, 0.0, 0.408248, 0.408248, 0.0, 0.288675, 0.333333, 0.0, 0.333333, 0.408248, 0.57735, 0.408248, 0.288675, 0.408248, 0.0, 0.288675, 0.333333, 0.666667, 0.288675, 0.57735, 0.333333, 0.408248, 0.333333, 0.0, 0.0, 0.0, 0.333333, 0.408248, 0.0, 0.57735, 0.333333, 0.333333, 0.408248, 0.408248, 0.0, 0.333333, 0.408248, 0.408248, 0.408248, 0.408248, 0.408248, 0.0, 0.0, 

127.0.0.1 - - [17/Jun/2021 05:03:39] "POST /predict HTTP/1.1" 200 -


Lila  [Pandas(Index=0, index=9931, _2=7097, genres='Drama|Horror|Action|', id=85695, imdb_id='tt0076644', original_title='Ruby', overview="Strange killings occur at Ruby's drive-in theatre, sixteen years after the murder of her gangster boyfriend.", poster_path='/wBkKWxXOmeKIR1VDI9JhzVm1Drd.jpg', production_companies='[]', release_date='1977-06-23', revenue='0', tagline="Christened in blood. Raised in sin. She's sweet sixteen, let the party begin!", title='Ruby', vote_average='4.6', vote_count='4', imdb_link='https://www.imdb.com/title/tt0076644', postername='Ruby Poster', poster_link='https://m.media-amazon.com/images/M/MV5BZDIyMDM1ODktMjRhYy00MTMyLWFmMDYtNTJlNTc3N2ZmZWViXkEyXkFqcGdeQXVyMTMxMTY0OTQ@._V1_UX182_CR0,0,182,268_AL__QL50.jpg', similarity=1.0), Pandas(Index=1, index=13815, _2=9525, genres='Horror|Action|Drama|', id=46658, imdb_id='tt0100273', original_title='Night of the Sharks', overview='David must fight for his life against the gangsters who killed his brother for a CD fi

127.0.0.1 - - [17/Jun/2021 05:03:40] "GET /static/poster_4.jpg HTTP/1.1" 200 -
